In [1]:
import pandas as pd 
AWM = pd.read_csv('/Users/5036365/Documents/Dravidian Lang Tech Data/misogyny/train-malayalam/train.csv')
len(AWM)

640

In [2]:
AWM_val = pd.read_csv('/Users/5036365/Documents/Dravidian Lang Tech Data/misogyny/dev-malayalam/dev.csv')
len(AWM_val)

160

In [3]:
AWM.columns

Index(['image_id', 'labels', 'transcriptions'], dtype='object')

In [4]:
AWM['labels'].value_counts()

labels
0    381
1    259
Name: count, dtype: int64

In [5]:
AWM['transcriptions'].nunique()

632

In [6]:
AWM = AWM.drop_duplicates(subset=['transcriptions'])
AWM = AWM.reset_index()

In [7]:
from transformers import AutoTokenizer
 
import torch
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import os

# Split the data into train and test sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(AWM['Text'], AWM['label'], test_size=0.2)
 

train_texts, train_labels,test_texts, test_labels = AWM['transcriptions'], AWM['labels'],AWM_val['transcriptions'], AWM_val['labels']
# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
 
# Tokenize dataset
def preprocess_function(examples):
    return tokenizer(examples.to_list(), truncation=True, padding="max_length", max_length=128)
 
train_encodings = preprocess_function(train_texts)
test_encodings = preprocess_function(test_texts)

/Users/5036365/Documents/text-recommendation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
# Convert to Hugging Face Dataset format, and include 'labels' in the Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels  # Ensure labels are included
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels  # Ensure labels are included
})


In [11]:
from transformers import AutoModelForSequenceClassification
 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=10,              # Number of epochs
    learning_rate=5e-6,
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/800 [00:00<?, ?it/s]

{'loss': 0.7159, 'grad_norm': 3.5516719818115234, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.12}
{'loss': 0.7224, 'grad_norm': 3.2044010162353516, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.25}
{'loss': 0.6997, 'grad_norm': 2.6323001384735107, 'learning_rate': 3.0000000000000004e-07, 'epoch': 0.38}
{'loss': 0.7023, 'grad_norm': 2.1803183555603027, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.5}
{'loss': 0.7105, 'grad_norm': 1.2324211597442627, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.62}
{'loss': 0.7066, 'grad_norm': 3.393435478210449, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.75}
{'loss': 0.7187, 'grad_norm': 2.9859845638275146, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.88}
{'loss': 0.7029, 'grad_norm': 2.8239495754241943, 'learning_rate': 8.000000000000001e-07, 'epoch': 1.0}


  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 0.7031892538070679, 'eval_accuracy': 0.39375, 'eval_macro_f1': 0.2825112107623318, 'eval_weighted_f1': 0.22247757847533628, 'eval_runtime': 1.4025, 'eval_samples_per_second': 114.078, 'eval_steps_per_second': 14.26, 'epoch': 1.0}
{'loss': 0.6909, 'grad_norm': 2.3745648860931396, 'learning_rate': 9.000000000000001e-07, 'epoch': 1.12}
{'loss': 0.6961, 'grad_norm': 1.328694224357605, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.25}
{'loss': 0.6998, 'grad_norm': 2.0456035137176514, 'learning_rate': 1.1e-06, 'epoch': 1.38}
{'loss': 0.6759, 'grad_norm': 7.160404682159424, 'learning_rate': 1.2000000000000002e-06, 'epoch': 1.5}
{'loss': 0.6781, 'grad_norm': 4.182975769042969, 'learning_rate': 1.3e-06, 'epoch': 1.62}
{'loss': 0.6913, 'grad_norm': 8.42760181427002, 'learning_rate': 1.4000000000000001e-06, 'epoch': 1.75}
{'loss': 0.6528, 'grad_norm': 5.943810939788818, 'learning_rate': 1.5e-06, 'epoch': 1.88}
{'loss': 0.6915, 'grad_norm': 7.6463541984558105, 'learning_rate': 

...

{'eval_loss': 0.7630288004875183, 'eval_accuracy': 0.73125, 'eval_macro_f1': 0.7255793546328427, 'eval_weighted_f1': 0.7339620477842926, 'eval_runtime': 1.3716, 'eval_samples_per_second': 116.652, 'eval_steps_per_second': 14.582, 'epoch': 8.0}
{'loss': 0.3114, 'grad_norm': 21.581546783447266, 'learning_rate': 2.5e-06, 'epoch': 8.12}
{'loss': 0.168, 'grad_norm': 20.375988006591797, 'learning_rate': 2.3333333333333336e-06, 'epoch': 8.25}
{'loss': 0.3832, 'grad_norm': 71.0948486328125, 'learning_rate': 2.166666666666667e-06, 'epoch': 8.38}
{'loss': 0.2531, 'grad_norm': 32.17427444458008, 'learning_rate': 2.0000000000000003e-06, 'epoch': 8.5}
{'loss': 0.3431, 'grad_norm': 5.899667263031006, 'learning_rate': 1.8333333333333333e-06, 'epoch': 8.62}
{'loss': 0.4439, 'grad_norm': 69.28955078125, 'learning_rate': 1.6666666666666667e-06, 'epoch': 8.75}
{'loss': 0.3019, 'grad_norm': 8.161149978637695, 'learning_rate': 1.5e-06, 'epoch': 8.88}
{'loss': 0.3572, 'grad_norm': 1.7894833087921143, 'learn

  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 0.7870129346847534, 'eval_accuracy': 0.725, 'eval_macro_f1': 0.7186700767263428, 'eval_weighted_f1': 0.7276374680306906, 'eval_runtime': 1.3667, 'eval_samples_per_second': 117.072, 'eval_steps_per_second': 14.634, 'epoch': 9.0}
{'loss': 0.3502, 'grad_norm': 47.56480026245117, 'learning_rate': 1.1666666666666668e-06, 'epoch': 9.12}
{'loss': 0.3661, 'grad_norm': 68.46880340576172, 'learning_rate': 1.0000000000000002e-06, 'epoch': 9.25}
{'loss': 0.2057, 'grad_norm': 3.247373580932617, 'learning_rate': 8.333333333333333e-07, 'epoch': 9.38}
{'loss': 0.2772, 'grad_norm': 50.52203369140625, 'learning_rate': 6.666666666666667e-07, 'epoch': 9.5}
{'loss': 0.2511, 'grad_norm': 15.838977813720703, 'learning_rate': 5.000000000000001e-07, 'epoch': 9.62}
{'loss': 0.3188, 'grad_norm': 9.75171184539795, 'learning_rate': 3.3333333333333335e-07, 'epoch': 9.75}
{'loss': 0.1455, 'grad_norm': 8.756784439086914, 'learning_rate': 1.6666666666666668e-07, 'epoch': 9.88}
{'loss': 0.3527, 'grad_norm

  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 0.7994685173034668, 'eval_accuracy': 0.725, 'eval_macro_f1': 0.717541325629915, 'eval_weighted_f1': 0.7272949767292569, 'eval_runtime': 1.3976, 'eval_samples_per_second': 114.479, 'eval_steps_per_second': 14.31, 'epoch': 10.0}
{'train_runtime': 358.6133, 'train_samples_per_second': 17.847, 'train_steps_per_second': 2.231, 'train_loss': 0.4957415705919266, 'epoch': 10.0}


In [13]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=10,              # Number of epochs
    per_device_train_batch_size=16,   # Batch size for training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/400 [00:00<?, ?it/s]

{'loss': 0.5491, 'grad_norm': 10.275177955627441, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.25}
{'loss': 0.5738, 'grad_norm': 7.985166072845459, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.5}
{'loss': 0.5791, 'grad_norm': 7.087610721588135, 'learning_rate': 3e-06, 'epoch': 0.75}
{'loss': 0.5254, 'grad_norm': 13.750337600708008, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.5623343586921692, 'eval_accuracy': 0.775, 'eval_macro_f1': 0.758348716227555, 'eval_weighted_f1': 0.7718283269004866, 'eval_runtime': 1.3104, 'eval_samples_per_second': 122.096, 'eval_steps_per_second': 7.631, 'epoch': 1.0}
{'loss': 0.4999, 'grad_norm': 13.740409851074219, 'learning_rate': 5e-06, 'epoch': 1.25}
{'loss': 0.5083, 'grad_norm': 24.48983383178711, 'learning_rate': 6e-06, 'epoch': 1.5}
{'loss': 0.5443, 'grad_norm': 21.156694412231445, 'learning_rate': 7.000000000000001e-06, 'epoch': 1.75}
{'loss': 0.5039, 'grad_norm': 29.844383239746094, 'learning_rate': 8.000000000000001e-06, 'epoch': 2.0}


  0%|          | 0/10 [00:00<?, ?it/s]

...

{'eval_loss': 0.8911710977554321, 'eval_accuracy': 0.78125, 'eval_macro_f1': 0.7757757757757757, 'eval_weighted_f1': 0.7832207207207208, 'eval_runtime': 1.2981, 'eval_samples_per_second': 123.261, 'eval_steps_per_second': 7.704, 'epoch': 8.0}
{'loss': 0.0812, 'grad_norm': 1.328125238418579, 'learning_rate': 3.3e-05, 'epoch': 8.25}
{'loss': 0.237, 'grad_norm': 14.304564476013184, 'learning_rate': 3.4000000000000007e-05, 'epoch': 8.5}
{'loss': 0.1622, 'grad_norm': 25.417861938476562, 'learning_rate': 3.5e-05, 'epoch': 8.75}
{'loss': 0.401, 'grad_norm': 29.40496826171875, 'learning_rate': 3.6e-05, 'epoch': 9.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.6801835894584656, 'eval_accuracy': 0.80625, 'eval_macro_f1': 0.800570946081782, 'eval_weighted_f1': 0.8077223473121306, 'eval_runtime': 1.3552, 'eval_samples_per_second': 118.064, 'eval_steps_per_second': 7.379, 'epoch': 9.0}
{'loss': 0.1879, 'grad_norm': 166.05685424804688, 'learning_rate': 3.7e-05, 'epoch': 9.25}
{'loss': 0.1109, 'grad_norm': 1.318762183189392, 'learning_rate': 3.8e-05, 'epoch': 9.5}
{'loss': 0.2646, 'grad_norm': 11.164958953857422, 'learning_rate': 3.9000000000000006e-05, 'epoch': 9.75}
{'loss': 0.2247, 'grad_norm': 0.19682539999485016, 'learning_rate': 4e-05, 'epoch': 10.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 1.1271089315414429, 'eval_accuracy': 0.775, 'eval_macro_f1': 0.7698209718670077, 'eval_weighted_f1': 0.7771579283887469, 'eval_runtime': 1.3144, 'eval_samples_per_second': 121.731, 'eval_steps_per_second': 7.608, 'epoch': 10.0}
{'train_runtime': 275.3088, 'train_samples_per_second': 23.247, 'train_steps_per_second': 1.453, 'train_loss': 0.3357498075067997, 'epoch': 10.0}


In [7]:
from transformers import AutoTokenizer
 
import torch
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import os
from transformers import BertTokenizer, BertForSequenceClassification

# Split the data into train and test sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(AWM['Text'], AWM['label'], test_size=0.2)
 

train_texts, train_labels,test_texts, test_labels = AWM['transcriptions'], AWM['labels'],AWM_val['transcriptions'], AWM_val['labels']
# Load tokenizer
model_name = "l3cube-pune/malayalam-bert"
tokenizer = BertTokenizer.from_pretrained(model_name)
 
# Tokenize dataset
def preprocess_function(examples):
    return tokenizer(examples.to_list(), truncation=True, padding="max_length", max_length=128)
 
train_encodings = preprocess_function(train_texts)
test_encodings = preprocess_function(test_texts)

/Users/5036365/Documents/text-recommendation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
# Convert to Hugging Face Dataset format, and include 'labels' in the Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels  # Ensure labels are included
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels  # Ensure labels are included
})


In [9]:
 
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/malayalam-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=10,              # Number of epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/790 [00:00<?, ?it/s]

{'loss': 0.6934, 'grad_norm': 0.24948449432849884, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.13}
{'loss': 0.6934, 'grad_norm': 0.4617324769496918, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.25}
{'loss': 0.6932, 'grad_norm': 0.08503407984972, 'learning_rate': 3e-06, 'epoch': 0.38}
{'loss': 0.6929, 'grad_norm': 0.23755939304828644, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.51}
{'loss': 0.6927, 'grad_norm': 0.24050067365169525, 'learning_rate': 5e-06, 'epoch': 0.63}
{'loss': 0.6924, 'grad_norm': 0.08457814902067184, 'learning_rate': 6e-06, 'epoch': 0.76}
{'loss': 0.6911, 'grad_norm': 0.2601985037326813, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.89}


  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 0.6884125471115112, 'eval_accuracy': 0.60625, 'eval_macro_f1': 0.377431906614786, 'eval_weighted_f1': 0.45763618677042806, 'eval_runtime': 1.3897, 'eval_samples_per_second': 115.135, 'eval_steps_per_second': 14.392, 'epoch': 1.0}
{'loss': 0.6892, 'grad_norm': 0.2505854368209839, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.01}
{'loss': 0.6837, 'grad_norm': 0.5139064192771912, 'learning_rate': 9e-06, 'epoch': 1.14}
{'loss': 0.6795, 'grad_norm': 0.5908999443054199, 'learning_rate': 1e-05, 'epoch': 1.27}
{'loss': 0.66, 'grad_norm': 0.6277580857276917, 'learning_rate': 1.1000000000000001e-05, 'epoch': 1.39}
{'loss': 0.6577, 'grad_norm': 0.5389516949653625, 'learning_rate': 1.2e-05, 'epoch': 1.52}
{'loss': 0.6619, 'grad_norm': 0.8741613626480103, 'learning_rate': 1.3000000000000001e-05, 'epoch': 1.65}
{'loss': 0.6491, 'grad_norm': 1.0870167016983032, 'learning_rate': 1.4000000000000001e-05, 'epoch': 1.77}
{'loss': 0.6329, 'grad_norm': 1.0668222904205322, 'learning_rate':

  0%|          | 0/20 [00:00<?, ?it/s]

...

{'eval_loss': 0.9701939821243286, 'eval_accuracy': 0.8, 'eval_macro_f1': 0.7953964194373402, 'eval_weighted_f1': 0.8019181585677749, 'eval_runtime': 1.3892, 'eval_samples_per_second': 115.174, 'eval_steps_per_second': 14.397, 'epoch': 8.0}
{'loss': 0.0066, 'grad_norm': 0.07300256937742233, 'learning_rate': 2.5862068965517244e-05, 'epoch': 8.1}
{'loss': 0.0063, 'grad_norm': 0.05681227892637253, 'learning_rate': 2.413793103448276e-05, 'epoch': 8.23}
{'loss': 0.0061, 'grad_norm': 0.14075367152690887, 'learning_rate': 2.2413793103448276e-05, 'epoch': 8.35}
{'loss': 0.0058, 'grad_norm': 0.05322113633155823, 'learning_rate': 2.0689655172413793e-05, 'epoch': 8.48}
{'loss': 0.0055, 'grad_norm': 0.06095152720808983, 'learning_rate': 1.896551724137931e-05, 'epoch': 8.61}
{'loss': 0.0053, 'grad_norm': 0.5879468321800232, 'learning_rate': 1.7241379310344828e-05, 'epoch': 8.73}
{'loss': 0.0051, 'grad_norm': 0.05100762099027634, 'learning_rate': 1.5517241379310346e-05, 'epoch': 8.86}
{'loss': 0.0723

  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 1.0263177156448364, 'eval_accuracy': 0.79375, 'eval_macro_f1': 0.7885885885885886, 'eval_weighted_f1': 0.7956081081081081, 'eval_runtime': 1.3766, 'eval_samples_per_second': 116.232, 'eval_steps_per_second': 14.529, 'epoch': 9.0}
{'loss': 0.1354, 'grad_norm': 0.06488047540187836, 'learning_rate': 1.206896551724138e-05, 'epoch': 9.11}
{'loss': 0.005, 'grad_norm': 0.050644032657146454, 'learning_rate': 1.0344827586206897e-05, 'epoch': 9.24}
{'loss': 0.071, 'grad_norm': 0.05173572525382042, 'learning_rate': 8.620689655172414e-06, 'epoch': 9.37}
{'loss': 0.0051, 'grad_norm': 0.05438721552491188, 'learning_rate': 6.896551724137932e-06, 'epoch': 9.49}
{'loss': 0.0049, 'grad_norm': 0.04680320993065834, 'learning_rate': 5.172413793103448e-06, 'epoch': 9.62}
{'loss': 0.0048, 'grad_norm': 0.05012199655175209, 'learning_rate': 3.448275862068966e-06, 'epoch': 9.75}
{'loss': 0.0049, 'grad_norm': 0.048814721405506134, 'learning_rate': 1.724137931034483e-06, 'epoch': 9.87}
{'loss': 0.00

  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 0.9885454177856445, 'eval_accuracy': 0.81875, 'eval_macro_f1': 0.8125934003796599, 'eval_weighted_f1': 0.8198114826931621, 'eval_runtime': 1.4072, 'eval_samples_per_second': 113.702, 'eval_steps_per_second': 14.213, 'epoch': 10.0}
{'train_runtime': 324.8987, 'train_samples_per_second': 19.452, 'train_steps_per_second': 2.432, 'train_loss': 0.27347213508018964, 'epoch': 10.0}


In [11]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=10,              # Number of epochs
    per_device_train_batch_size=4,   # Batch size for training
    per_device_eval_batch_size=4,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/1580 [00:00<?, ?it/s]

{'loss': 0.128, 'grad_norm': 0.6564546823501587, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.06}
{'loss': 0.2372, 'grad_norm': 0.5993016958236694, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.13}
{'loss': 0.1448, 'grad_norm': 3.769517183303833, 'learning_rate': 3e-06, 'epoch': 0.19}
{'loss': 0.1272, 'grad_norm': 1.7521510124206543, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.25}
{'loss': 0.135, 'grad_norm': 0.6707231998443604, 'learning_rate': 5e-06, 'epoch': 0.32}
{'loss': 0.2238, 'grad_norm': 7.000926971435547, 'learning_rate': 6e-06, 'epoch': 0.38}
{'loss': 0.1217, 'grad_norm': 0.6378626823425293, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.44}
{'loss': 0.1657, 'grad_norm': 0.8278731107711792, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.51}
{'loss': 0.22, 'grad_norm': 0.6149020791053772, 'learning_rate': 9e-06, 'epoch': 0.57}
{'loss': 0.1631, 'grad_norm': 0.5099255442619324, 'learning_rate': 1e-05, 'epoch': 0.63}
{'loss': 0.1041, 'grad_norm': 0.50

  0%|          | 0/40 [00:00<?, ?it/s]

{'eval_loss': 0.5982922315597534, 'eval_accuracy': 0.7875, 'eval_macro_f1': 0.7797570850202429, 'eval_weighted_f1': 0.7885323886639677, 'eval_runtime': 1.5718, 'eval_samples_per_second': 101.795, 'eval_steps_per_second': 25.449, 'epoch': 1.0}
{'loss': 0.143, 'grad_norm': 0.42859432101249695, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.01}
{'loss': 0.0732, 'grad_norm': 0.46448010206222534, 'learning_rate': 1.7000000000000003e-05, 'epoch': 1.08}
{'loss': 0.1273, 'grad_norm': 0.3940146565437317, 'learning_rate': 1.8e-05, 'epoch': 1.14}
{'loss': 0.06, 'grad_norm': 0.3577873110771179, 'learning_rate': 1.9e-05, 'epoch': 1.2}
{'loss': 0.0555, 'grad_norm': 0.3156800866127014, 'learning_rate': 2e-05, 'epoch': 1.27}
{'loss': 0.0493, 'grad_norm': 0.32305166125297546, 'learning_rate': 2.1e-05, 'epoch': 1.33}
{'loss': 0.0436, 'grad_norm': 0.3015062212944031, 'learning_rate': 2.2000000000000003e-05, 'epoch': 1.39}
{'loss': 0.1301, 'grad_norm': 3.7693095207214355, 'learning_rate': 2.300000000

  0%|          | 0/40 [00:00<?, ?it/s]

...

{'eval_loss': 1.013304352760315, 'eval_accuracy': 0.84375, 'eval_macro_f1': 0.837655749015788, 'eval_weighted_f1': 0.8443397662242786, 'eval_runtime': 1.6357, 'eval_samples_per_second': 97.815, 'eval_steps_per_second': 24.454, 'epoch': 8.0}
{'loss': 0.0016, 'grad_norm': 0.017357761040329933, 'learning_rate': 1.4351851851851853e-05, 'epoch': 8.04}
{'loss': 0.0014, 'grad_norm': 0.016289303079247475, 'learning_rate': 1.388888888888889e-05, 'epoch': 8.1}
{'loss': 0.0014, 'grad_norm': 0.016514286398887634, 'learning_rate': 1.3425925925925928e-05, 'epoch': 8.16}
{'loss': 0.0014, 'grad_norm': 0.02285444550216198, 'learning_rate': 1.2962962962962962e-05, 'epoch': 8.23}
{'loss': 0.0014, 'grad_norm': 0.01806597039103508, 'learning_rate': 1.25e-05, 'epoch': 8.29}
{'loss': 0.0015, 'grad_norm': 0.018767623230814934, 'learning_rate': 1.2037037037037037e-05, 'epoch': 8.35}
{'loss': 0.0013, 'grad_norm': 0.016672920435667038, 'learning_rate': 1.1574074074074075e-05, 'epoch': 8.42}
{'loss': 0.0014, 'gra

  0%|          | 0/40 [00:00<?, ?it/s]

{'eval_loss': 1.069270133972168, 'eval_accuracy': 0.8375, 'eval_macro_f1': 0.8315789473684211, 'eval_weighted_f1': 0.8382894736842106, 'eval_runtime': 1.6118, 'eval_samples_per_second': 99.265, 'eval_steps_per_second': 24.816, 'epoch': 9.0}
{'loss': 0.0012, 'grad_norm': 0.015817783772945404, 'learning_rate': 6.944444444444445e-06, 'epoch': 9.05}
{'loss': 0.0013, 'grad_norm': 0.015209618955850601, 'learning_rate': 6.481481481481481e-06, 'epoch': 9.11}
{'loss': 0.0011, 'grad_norm': 0.013071810826659203, 'learning_rate': 6.0185185185185185e-06, 'epoch': 9.18}
{'loss': 0.0012, 'grad_norm': 0.014737587422132492, 'learning_rate': 5.555555555555556e-06, 'epoch': 9.24}
{'loss': 0.0013, 'grad_norm': 0.013790599070489407, 'learning_rate': 5.092592592592592e-06, 'epoch': 9.3}
{'loss': 0.0011, 'grad_norm': 0.013815181329846382, 'learning_rate': 4.6296296296296296e-06, 'epoch': 9.37}
{'loss': 0.0011, 'grad_norm': 0.012853747233748436, 'learning_rate': 4.166666666666667e-06, 'epoch': 9.43}
{'loss': 

  0%|          | 0/40 [00:00<?, ?it/s]

{'eval_loss': 1.1403220891952515, 'eval_accuracy': 0.825, 'eval_macro_f1': 0.819471308833011, 'eval_weighted_f1': 0.8261847195357834, 'eval_runtime': 1.6195, 'eval_samples_per_second': 98.798, 'eval_steps_per_second': 24.7, 'epoch': 10.0}
{'train_runtime': 485.9254, 'train_samples_per_second': 13.006, 'train_steps_per_second': 3.252, 'train_loss': 0.10091292235021822, 'epoch': 10.0}
